In [37]:
import numpy as np
import time
import warnings
import pandas as pd
import pickle

import quantpy as qp
from quantpy.routines import _left_inv

from collections import defaultdict, Counter
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore', category=np.ComplexWarning)

In [61]:
pauli = qp.routines.generate_pauli(2)
lambdas = pauli / np.sqrt(2)
np.trace(lambdas[4] @ lambdas[4])

(1.9999999999999996+0j)

In [86]:
state = qp.qobj.GHZ(1)
tmg = qp.StateTomograph(state)
tmg.experiment(1000, 'proj-set')
state_hat = tmg.point_estimate()
state_hat

Quantum object
array([[0.5    -0.j   , 0.49986+0.012j],
       [0.49986-0.012j, 0.5    +0.j   ]])

In [92]:
povm_matrix = np.reshape(tmg.povm_matrix, (-1, tmg.povm_matrix.shape[-1])) * 2 ** tmg.state.n_qubits
inversed_povm = _left_inv(povm_matrix).reshape(
    (-1, tmg.povm_matrix.shape[0], tmg.povm_matrix.shape[1]))
measurement_ratios = tmg.n_measurements.sum() / tmg.n_measurements
povm_matrix.shape

(6, 4)

In [88]:
np.max(inversed_povm, axis=-1) - np.min(inversed_povm, axis=-1)

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [90]:
c_alpha = np.sum((np.max(inversed_povm, axis=-1) - np.min(inversed_povm, axis=-1)) ** 2 
                 * measurement_ratios[None, :], axis=-1) + 1e-15
c_alpha

array([0., 3., 3., 3.])

In [50]:
povm_matrix.sum(axis=0)

array([6., 0., 0., 0.])

In [51]:
povm_matrix @ state.bloch

array([1. , 0. , 0.5, 0.5, 0.5, 0.5])

In [24]:
povm_matrix = qp.measurements.generate_measurement_matrix('proj-set', 2) * 4
povm_matrix.shape

(9, 4, 16)

In [25]:
povm_matrix[0][0]

array([1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
qp.Qobj(povm_matrix[0][0])

Quantum object
array([[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j]])

In [42]:
xpos = qp.Qobj([0.5, 0.5, 0, 0])
xpos2 = xpos.kron(xpos)
xpos2

Quantum object
array([[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j],
       [0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j]])

In [23]:
np.trace(xpos2.matrix @ lambdas[1]) / 2 / np.sqrt(2)

(0.24999999999999994+0j)